## **Data Science Workplow**

Berliner Hochschule für Technik

News Reader project - Cat news

Polina Kozyr

In this project I extracted news about cats from cat news webpage http://catdailynews.com/ and made topic classification and sentiment analysis of these news. For topic classification I trained classificator to predict topic of cat news. For sentiment analysis I just used already pre-trained model to classify news on positive and negative ones. Also I made a web-application to make an end-to-end solution for the cat news reader.

Project content:

1. Extraction of news from website - creation of cat news dataset
2. Topic classification
        1) Cleaning of news texts
        2) Formation of topic classification system stages
        2) Classification for humorous or not humorous articles
        3) Classification for other categories
        4) Сombining results from two classifications
3. Sentiment analysis
4. Web-application creation

I also added code that I used to create a web-application in the end of this NoteBook. Web-application extract recent news from http://catdailynews.com/, proccess them, predict their categories and sentiment and print results. 

Link for web-application: https://mybinder.org/v2/gh/polinatrump/news_reader_webapp_demo/HEAD?urlpath=%2Fvoila%2Frender%2FDemo.ipynb

Web-app loading can take some time, sometimes really long.

## 1. Extraction of news from website - creation of cat news dataset

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

import string
import re
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

import numpy as np

import pickle

### Get title and link of news from pages that have list of news

In [60]:
news_data = []
for subpage_index in range(1, 200):
    page = "http://catdailynews.com/page/" + str(subpage_index) + "/"
#     print(page)
    protocol_list = requests.get(page).text
    soup = BeautifulSoup(protocol_list, 'lxml')
    
    for ai in soup.find_all('a'):
        if ai.div:
            title = ai.div.text
            link = ai.get('href')
#             print(title)
#             print(link)
            news_data.append((title, link))


In [61]:
print(str(len(news_data)) + ' news links were extracted from catnews webpages.')

1990 news links were extracted from catnews webpages.


### Get  text, category and date from each news link page and write news data to the csv file

In [62]:
n = 0

for idx, i in enumerate(news_data):
#     n +=1
#     print(n)
    
    protocol_list = requests.get(i[1]).text
    soup = BeautifulSoup(protocol_list, 'lxml')
    
    text_parts = soup.find_all('p')
    text = ''
    for p in text_parts:
        text += p.text
    
    for divi in soup.find_all('div'):
        if divi.get("class") == ['pull-left']:
            date = divi.text.split('  ')[0][:-2]
                
    categories = []
    for divi in soup.find_all('div'):
        if divi.get("class") == ['category-tag']:
            all_a = divi.find_all('a')
            for a in all_a:
                if a.text[0].isupper():
                    categories.append(a.text)
            categories = ', '.join(categories)
            
    with open('news200.csv', 'a', newline='', encoding="utf-8") as csvfile:
        spamwriter = csv.writer(csvfile, delimiter='|')
        spamwriter.writerow([i[0], i[1], date, text, categories])
        

## 2. Topic classification

### Import cat news data

In [64]:
data = pd.read_csv('news200.csv', sep='|', names=['title', 'link', 'date', 'text', 'category'])
data.head()

,title,link,date,text,category
0,Indiana Jones and a Cat,http://catdailynews.com/2023/06/indiana-jones-...,"June 30, 2023",With the latest Indiana Jones movie out in the...,"Humor, Video"
1,Interesting Cat Facts,http://catdailynews.com/2023/06/interesting-ca...,"June 29, 2023",Many people love cats. However what makes cat...,Humor
2,Cat Gets “Jailed” For Being Too Daring,http://catdailynews.com/2023/06/cat-gets-jaile...,"June 28, 2023",Alice likes traveling in a van on the road wit...,"Humor, Travel"
3,Toys to Keep Your Cat Happy,http://catdailynews.com/2023/06/toys-to-keep-y...,"June 27, 2023","Cats need lots of attention, except when they ...",Humor
4,The Evolution of Cats,http://catdailynews.com/2023/06/the-evolution-...,"June 26, 2023",People love cats so it’s no surprise that peop...,"Science, Egypt"


In [65]:
print("Number of news: ", data.shape[0])

Number of news:  1990


### 1) Preproccessing and cleaning of news data

In [4]:
def remove_stopwords(text):
    return " ".join([w if w not in eng_stopwords else ' ' for w in text.split(' ')])

def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

def remove_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text, flags=re.I)

def lemmatize_text(text):
    return " ".join([lemma.lemmatize(word.lower()) for word in text.split(' ')])

lemma = WordNetLemmatizer()

eng_stopwords = stopwords.words("english")
eng_stopwords.extend(['…', '«', '»', '...'])

In [5]:
data['cleaned_text'] = data['text'].apply(lambda text: lemmatize_text((remove_multiple_spaces(remove_numbers(remove_stopwords(remove_punctuation(text)))))))
data['cleaned_text']

0       with latest indiana jones movie theater people...
1       many people love cat however make cat even lov...
2       alice like traveling van road cat unfortunatel...
3       cat need lot attention except want left alone ...
4       people love cat it’s surprise people also stud...
                              ...                        
1985    even hollywood celebrity stay inside prevent s...
1986    with people unable watch live sporting event c...
1987    if you’ve ever taken picture cat may startled ...
1988    erin merryn daughter ginger tabby named carrot...
1989    in upton merseyside england employee sainsbury...
Name: cleaned_text, Length: 1990, dtype: object

Print unique category sets

In [6]:
print(data['category'].unique(), len(data['category'].unique()))

['Humor, Video' 'Humor' 'Humor, Travel' 'Science, Egypt'
 'Games, Humor, Technology' 'Humor, England' 'Humor, Technology'
 'Homeless cat' 'Humor, Science' 'Games, Technology' 'Games, Humor'
 'Science' 'Humor, Show business' 'Humor, Military' 'Humor, Iceland'
 'Humor, Japan' 'Technology' 'Humor, Morocco' 'Health' 'Uncategorized'
 'Humor, Travel, Thailand' 'Health, Medical' 'Humor, Turkey'
 'Homeless cat, Japan' 'Humor, Travel, Poland' 'Homeless cat, Turkey'
 'Homeless cat, Humor' 'Humor, India' 'Humor, Thailand'
 'Humor, Netherlands, Ukraine' 'Humor, Singapore' 'Health, England'
 'Humor, Travel, Japan' 'Humor, Vietnam' 'Homeless cat, Video'
 'Humor, Canada' 'Homeless cat, Humor, Show business' 'Humor, Australia'
 'Humor, Peru' 'Humor, Ukraine' 'Humor, Sports, Brazil'
 'Homeless cat, Humor, Video' 'Humor, Italy' 'Health, Humor'
 'Health, Science' 'Medical' 'Health, Homeless cat'
 'Humor, Video, England' 'Humor, Greece' 'Crazy cat people, Humor'
 'Homeless cat, Ukraine' 'Financial' 'Show 

As you can see articles can have more than one category. 

Furthermore, country name can also be the last category in category set of article. In this project I will not use counry as a category, that is why I remove all country names from category sets of cat news:

In [7]:
countries = ['England', 'Vietnam', 'Afghanistan', 'Aland Islands', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia, Plurinational State of', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', 'Congo, The Democratic Republic of the', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Heard Island and McDonald Islands', 'Holy See (Vatican City State)', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Republic of', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Democratic People's Republic of", 'Korea, Republic of', 'Kuwait', 'Kyrgyzstan', "Lao People's Democratic Republic", 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao', 'Macedonia, Republic of', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', 'Mauritius', 'Mayotte', 'Mexico', 'Micronesia, Federated States of', 'Moldova, Republic of', 'Monaco', 'Mongolia', 'Montenegro', 'Montserrat', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'Niue', 'Norfolk Island', 'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestinian Territory, Occupied', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Pitcairn', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Réunion', 'Romania', 'Russian Federation', 'Rwanda', 'Saint Barthélemy', 'Saint Helena, Ascension and Tristan da Cunha', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Martin (French part)', 'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Georgia and the South Sandwich Islands', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'South Sudan', 'Svalbard and Jan Mayen', 'Swaziland', 'Sweden', 'Switzerland', 'Syrian Arab Republic', 'Taiwan, Province of China', 'Tajikistan', 'Tanzania, United Republic of', 'Thailand', 'Timor-Leste', 'Togo', 'Tokelau', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'United States Minor Outlying Islands', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, Bolivarian Republic of', 'Viet Nam', 'Virgin Islands, British', 'Virgin Islands, U.S.', 'Wallis and Futuna', 'Yemen', 'Zambia', 'Zimbabwe']

def remove_country_category(category_set):
    categories = category_set.split(', ')
    categories = [category for category in categories if category not in countries]
    categories = ', '.join(categories)
    return categories

data['categories_without_countries'] = data['category'].apply(lambda category_set: remove_country_category(category_set))

### 2) Formation of topic classification system stages

Now let's have a look at "Humor" category. We can see that this category can be seen in the most of news:

In [8]:
print(str(len([c_set for c_set in data['categories_without_countries'] if "Humor" in c_set])) + ' out of ' + str(len(data)) + ' news have "Humor" category in their category set.')

1515 out of 1990 news have "Humor" category in their category set.


Number of all possible category sets:

In [9]:
data['categories_without_countries'].value_counts()[:20]

Humor                                     754
Crazy cat people, Humor                   145
Health                                    123
Humor, Show business                       93
Humor, Video                               82
Homeless cat                               72
Humor, Technology                          62
Crazy cat people, Humor, Show business     47
Humor, Science                             45
Crazy cat people                           39
Health, Medical                            36
Homeless cat, Humor                        31
Science                                    29
Health, Humor                              26
Humor, Travel                              23
Crazy cat people, Health                   19
Crazy cat people, Humor, Technology        15
Games, Humor, Technology                   15
Crazy cat people, Homeless cat             15
Crazy cat people, Humor, Video             13
Name: categories_without_countries, dtype: int64

From the above cell we can see that the most popular category set consists only from category "Humor". After "Humor"we can see category sets that contain "Humor" plus some other category. 

I will reduce the amount of categories that i will try to predict. I take for classification task the following categories: "Humor", "Health", "Video", "Homeless cat", "Show business", "Technology", "Science". All other categories i add to the new category "No category":

In [10]:
def reduce_dataset(category_set):
    new_categories = []
    categories = category_set.split(', ')
    for category in categories:
        if category == 'Humor':
            new_categories.append(category)
            break
    for category in categories:
        if category in ['Health', 'Video', 'Homeless cat', 'Show business', 'Technology', 'Science']:
            new_categories.append(category)
            break
    if not new_categories:
        new_categories.append('No category')
    categories = ', '.join(new_categories)
    return categories



In [11]:
dataset = data.copy()

In [12]:
dataset['cleaned_category_sets'] = dataset['categories_without_countries'].apply(lambda category_set: reduce_dataset(category_set))

In [13]:
dataset.head()

,title,link,date,text,category,cleaned_text,categories_without_countries,cleaned_category_sets
0,Indiana Jones and a Cat,http://catdailynews.com/2023/06/indiana-jones-...,"June 30, 2023",With the latest Indiana Jones movie out in the...,"Humor, Video",with latest indiana jones movie theater people...,"Humor, Video","Humor, Video"
1,Interesting Cat Facts,http://catdailynews.com/2023/06/interesting-ca...,"June 29, 2023",Many people love cats. However what makes cat...,Humor,many people love cat however make cat even lov...,Humor,Humor
2,Cat Gets “Jailed” For Being Too Daring,http://catdailynews.com/2023/06/cat-gets-jaile...,"June 28, 2023",Alice likes traveling in a van on the road wit...,"Humor, Travel",alice like traveling van road cat unfortunatel...,"Humor, Travel",Humor
3,Toys to Keep Your Cat Happy,http://catdailynews.com/2023/06/toys-to-keep-y...,"June 27, 2023","Cats need lots of attention, except when they ...",Humor,cat need lot attention except want left alone ...,Humor,Humor
4,The Evolution of Cats,http://catdailynews.com/2023/06/the-evolution-...,"June 26, 2023",People love cats so it’s no surprise that peop...,"Science, Egypt",people love cat it’s surprise people also stud...,Science,Science




Cat content is funny so it is not suprising that there are so many humorous articles in the dataset. I decided to devide my classification system for two steps:

1. Classify news on humorous or not humorous
2. Classify news on remaining categories
3. Then I will combine results from both classifications and evaluate them.

I will save some data for testing of the final two stages system (humor classification and other categories classification) proportionally. 

In [14]:
dataset['cleaned_category_sets'].value_counts()[:20]

Humor                   994
Health                  215
Humor, Show business    158
Homeless cat            107
Humor, Video            101
Humor, Technology       101
No category              80
Humor, Science           57
Humor, Homeless cat      54
Humor, Health            50
Science                  39
Technology               19
Show business            13
Video                     2
Name: cleaned_category_sets, dtype: int64

In [15]:
threshold_humor = len(dataset[dataset['cleaned_category_sets'] == 'Humor']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Humor']) * 0.25)
final_test_dataset_humor = dataset[dataset['cleaned_category_sets'] == 'Humor'][threshold_humor:]
working_dataset_humor = dataset[dataset['cleaned_category_sets'] == 'Humor'][:threshold_humor]

#threshold_humor_crazy_cat_people = len(dataset[dataset['cleaned_category_sets'] == 'Humor, Crazy cat people']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Humor, Crazy cat people']) * 0.25)
#final_test_dataset_humor_crazy_cat_people = dataset[dataset['cleaned_category_sets'] == 'Humor, Crazy cat people'][threshold_humor_crazy_cat_people:]
#working_dataset_humor_crazy_cat_people = dataset[dataset['cleaned_category_sets'] == 'Humor, Crazy cat people'][:threshold_humor_crazy_cat_people]

threshold_health = len(dataset[dataset['cleaned_category_sets'] == 'Health']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Health']) * 0.25)
final_test_dataset_health = dataset[dataset['cleaned_category_sets'] == 'Health'][threshold_health:]
working_dataset_health = dataset[dataset['cleaned_category_sets'] == 'Health'][:threshold_health]

#threshold_crazy_cat_people = len(dataset[dataset['cleaned_category_sets'] == 'Crazy cat people']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Crazy cat people']) * 0.25)
#final_test_dataset_crazy_cat_people = dataset[dataset['cleaned_category_sets'] == 'Crazy cat people'][threshold_crazy_cat_people:]
#working_dataset_crazy_cat_people = dataset[dataset['cleaned_category_sets'] == 'Crazy cat people'][:threshold_crazy_cat_people]

threshold_humor_show_business = len(dataset[dataset['cleaned_category_sets'] == 'Humor, Show business']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Humor, Show business']) * 0.25)
final_test_dataset_humor_show_business = dataset[dataset['cleaned_category_sets'] == 'Humor, Show business'][threshold_humor_show_business:]
working_dataset_humor_show_business = dataset[dataset['cleaned_category_sets'] == 'Humor, Show business'][:threshold_humor_show_business]

threshold_homeless_cat = len(dataset[dataset['cleaned_category_sets'] == 'Homeless cat']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Homeless cat']) * 0.25)
final_test_dataset_homeless_cat = dataset[dataset['cleaned_category_sets'] == 'Homeless cat'][threshold_homeless_cat:]
working_dataset_homeless_cat = dataset[dataset['cleaned_category_sets'] == 'Homeless cat'][:threshold_homeless_cat]

threshold_humor_video = len(dataset[dataset['cleaned_category_sets'] == 'Humor, Video']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Humor, Video']) * 0.25)
final_test_dataset_humor_video = dataset[dataset['cleaned_category_sets'] == 'Humor, Video'][threshold_humor_video:]
working_dataset_humor_video = dataset[dataset['cleaned_category_sets'] == 'Humor, Video'][:threshold_humor_video]

threshold_humor_technology = len(dataset[dataset['cleaned_category_sets'] == 'Humor, Technology']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Humor, Technology']) * 0.25)
final_test_dataset_humor_technology = dataset[dataset['cleaned_category_sets'] == 'Humor, Technology'][threshold_humor_technology:]
working_dataset_humor_technology = dataset[dataset['cleaned_category_sets'] == 'Humor, Technology'][:threshold_humor_technology]

threshold_humor_science = len(dataset[dataset['cleaned_category_sets'] == 'Humor, Science']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Humor, Science']) * 0.25)
final_test_dataset_humor_science = dataset[dataset['cleaned_category_sets'] == 'Humor, Science'][threshold_humor_science:]
working_dataset_humor_science = dataset[dataset['cleaned_category_sets'] == 'Humor, Science'][:threshold_humor_science]

threshold_humor_homeless_cat = len(dataset[dataset['cleaned_category_sets'] == 'Humor, Homeless cat']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Humor, Homeless cat']) * 0.25)
final_test_dataset_humor_homeless_cat = dataset[dataset['cleaned_category_sets'] == 'Humor, Homeless cat'][threshold_humor_homeless_cat:]
working_dataset_humor_homeless_cat = dataset[dataset['cleaned_category_sets'] == 'Humor, Homeless cat'][:threshold_humor_homeless_cat]

threshold_humor_health = len(dataset[dataset['cleaned_category_sets'] == 'Humor, Health']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Humor, Health']) * 0.25)
final_test_dataset_humor_health = dataset[dataset['cleaned_category_sets'] == 'Humor, Health'][threshold_humor_health:]
working_dataset_humor_health = dataset[dataset['cleaned_category_sets'] == 'Humor, Health'][:threshold_humor_health]

threshold_no_category = len(dataset[dataset['cleaned_category_sets'] == 'No category']) - round(len(dataset[dataset['cleaned_category_sets'] == 'No category']) * 0.25)
final_test_dataset_no_category = dataset[dataset['cleaned_category_sets'] == 'No category'][threshold_no_category:]
working_dataset_no_category = dataset[dataset['cleaned_category_sets'] == 'No category'][:threshold_no_category]


threshold_science = len(dataset[dataset['cleaned_category_sets'] == 'Science']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Science']) * 0.25)
final_test_dataset_science = dataset[dataset['cleaned_category_sets'] == 'Science'][threshold_science:]
working_dataset_science = dataset[dataset['cleaned_category_sets'] == 'Science'][:threshold_science]

threshold_technology = len(dataset[dataset['cleaned_category_sets'] == 'Technology']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Technology']) * 0.25)
final_test_dataset_technology = dataset[dataset['cleaned_category_sets'] == 'Technology'][threshold_technology:]
working_dataset_technology = dataset[dataset['cleaned_category_sets'] == 'Technology'][:threshold_technology]

threshold_show_business = len(dataset[dataset['cleaned_category_sets'] == 'Show business']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Show business']) * 0.25)
final_test_dataset_show_business = dataset[dataset['cleaned_category_sets'] == 'Show business'][threshold_show_business:]
working_dataset_show_business = dataset[dataset['cleaned_category_sets'] == 'Show business'][:threshold_show_business]

threshold_video = len(dataset[dataset['cleaned_category_sets'] == 'Video']) - round(len(dataset[dataset['cleaned_category_sets'] == 'Video']) * 0.25)
final_test_dataset_video = dataset[dataset['cleaned_category_sets'] == 'Video'][threshold_video:]
working_dataset_video = dataset[dataset['cleaned_category_sets'] == 'Video'][:threshold_video]



final_test_frames = [final_test_dataset_humor, final_test_dataset_health,  
                   final_test_dataset_show_business, final_test_dataset_homeless_cat, final_test_dataset_humor_video, final_test_dataset_humor_technology, 
                    final_test_dataset_humor_science, final_test_dataset_humor_homeless_cat, final_test_dataset_humor_health, final_test_dataset_no_category,
                    final_test_dataset_science, final_test_dataset_technology, final_test_dataset_show_business, final_test_dataset_video]
final_test_dataset = pd.concat(final_test_frames)


working_dataset = [working_dataset_humor, working_dataset_health, 
                   working_dataset_humor_show_business, working_dataset_homeless_cat, working_dataset_humor_video, working_dataset_humor_technology, 
                    working_dataset_humor_science, working_dataset_humor_homeless_cat, working_dataset_humor_health, working_dataset_no_category,
                    working_dataset_science, working_dataset_technology, working_dataset_show_business, working_dataset_video]
working_dataset = pd.concat(working_dataset)

I will continue working with this dataset:

In [16]:
working_dataset['cleaned_category_sets'].value_counts()

Humor                   746
Health                  161
Humor, Show business    118
Homeless cat             80
Humor, Video             76
Humor, Technology        76
No category              60
Humor, Science           43
Humor, Homeless cat      40
Humor, Health            38
Science                  29
Technology               14
Show business            10
Video                     2
Name: cleaned_category_sets, dtype: int64

In [17]:
X_train, X_test, y_train, y_test = train_test_split(working_dataset['cleaned_text'], working_dataset['cleaned_category_sets'], test_size=0.2, random_state=42)

### 2) Classification for humorous or not humorous articles

### Features engineering

For classification for humorous or not humorous articles I make labels "Humor" or "not Humor" according each article category set.

In [18]:
def remove_categories_exept_humor(category_set):
    if "Humor" in category_set:
        return "Humor"
    else:
        return "not Humor"

In [19]:
working_dataset['category_humor_task'] = working_dataset['cleaned_category_sets'].apply(lambda category_set: remove_categories_exept_humor(category_set))

In [20]:
print('Number of humorous news: ', len(working_dataset[working_dataset['category_humor_task'] == "Humor"]))
print('Number of not humorous news: ', len(working_dataset[working_dataset['category_humor_task'] == "not Humor"]))

Number of humorous news:  1137
Number of not humorous news:  356


I want to make number of samples for each class similar for training

In [21]:
humor_train_dataset = working_dataset[working_dataset['category_humor_task'] == "Humor"][:500]
humor_test_dataset = working_dataset[working_dataset['category_humor_task'] == "Humor"][500:]
humor_train_dataset = pd.concat([humor_train_dataset, working_dataset[working_dataset['category_humor_task'] == "not Humor"]])

In [22]:
X_train, X_test, y_train, y_test = train_test_split(humor_train_dataset['cleaned_text'], humor_train_dataset['category_humor_task'], test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

### Choose the right model

Let's look at performance of two models: SGDClassifier (linear models with stochastic gradient descent) and KNeighborsClassifier (k-nearest neighbors):

In [23]:
sgd_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42))])

knb_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knb_clf', KNeighborsClassifier(n_neighbors=10))])

sgd_clf.fit(X_train, y_train)
knb_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('knb_clf', KNeighborsClassifier(n_neighbors=10))])

#### SGDClassifier results

In [24]:
predicted_sgd = sgd_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

       Humor       0.84      0.74      0.79       110
   not Humor       0.62      0.76      0.68        62

    accuracy                           0.74       172
   macro avg       0.73      0.75      0.73       172
weighted avg       0.76      0.74      0.75       172



#### KNeighborsClassifier

In [25]:
predicted_knb = knb_clf.predict(X_test)
print(metrics.classification_report(predicted_knb, y_test))

              precision    recall  f1-score   support

       Humor       0.91      0.70      0.79       124
   not Humor       0.51      0.81      0.63        48

    accuracy                           0.73       172
   macro avg       0.71      0.76      0.71       172
weighted avg       0.80      0.73      0.75       172



SGDClassifier has better results than KNeighborsClassifier, that's why I will use SGDClassifier for the further work.

### Hyperparameters optimization

Next I do hyperparameters optimization for SGDClassifier using 5-fold cross-validation:

In [26]:
sgd_clf = Pipeline(steps=[
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42))])

parameters = {
    'sgd_clf__loss': ['hinge', 'modified_huber'], 
    'sgd_clf__penalty': ['l2', 'l1', 'elasticnet']
}

clf = GridSearchCV(sgd_clf, parameters, n_jobs=5)
clf.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('sgd_clf',
                                        SGDClassifier(random_state=42))]),
             n_jobs=5,
             param_grid={'sgd_clf__loss': ['hinge', 'modified_huber'],
                         'sgd_clf__penalty': ['l2', 'l1', 'elasticnet']})

In [27]:
print("Best parameter (CV score=%0.3f):" % clf.best_score_)
print(clf.best_params_)

Best parameter (CV score=0.773):
{'sgd_clf__loss': 'hinge', 'sgd_clf__penalty': 'l2'}


### Humorous articles classification results

I train our SGDClassifier model again with the best hyperparameters:

In [29]:
humor_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42, loss='hinge', penalty='l2'))])


humor_clf.fit(X_train, y_train)

predicted_humorous_clf = humor_clf.predict(X_val)
print(metrics.classification_report(predicted_humorous_clf, y_val))


              precision    recall  f1-score   support

       Humor       0.81      0.81      0.81        91
   not Humor       0.63      0.63      0.63        46

    accuracy                           0.75       137
   macro avg       0.72      0.72      0.72       137
weighted avg       0.75      0.75      0.75       137



Save model as pickle file:

In [30]:
with open("humor_clf.pkl", "wb") as file: 
    pickle.dump(humor_clf, file) 

The F1 score for predicting humorous news on validation dataset about cats was 0.81, while for predicting non-humorous news about cats it was 0.63. Average F1-score is 0.72.

### 3) Classification for other categories

### Features engineering

I replace "Humor" category from each article set with "No category". So now the task is to predict all categories exept "Humor" plus "No category" category.

In [31]:
def remove_humor_category(categories):
    categories = categories.split(', ')
    if 'Humor' in categories and len(categories) > 1:
        categories.remove('Humor')
    elif 'Humor' in categories and len(categories) == 1:
        categories = [i.replace('Humor', 'No category') for i in categories]
    categories = ', '.join(categories)
    return categories


In [32]:
working_dataset['category_other_categ_task'] = working_dataset['cleaned_category_sets'].apply(lambda category_set: remove_humor_category(category_set))

In this dataset we have too many samples that belong to "No category" class. First I decided to reduce their amount to make dataset more balanced, but it just made classification results worse. Then I decided to keep the amount of data from different categories as it is.

In [33]:
working_dataset['category_other_categ_task'].value_counts()

No category      806
Health           199
Show business    128
Homeless cat     120
Technology        90
Video             78
Science           72
Name: category_other_categ_task, dtype: int64

In [34]:
X_train, X_test, y_train, y_test = train_test_split(working_dataset['cleaned_text'], working_dataset['category_other_categ_task'], test_size=0.2, random_state=23)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

### Choose the right model

For this task let's also look at performance of two models: SGDClassifier (linear models with stochastic gradient descent) and KNeighborsClassifier (k-nearest neighbors):

In [35]:
sgd_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42))])

knb_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knb_clf', KNeighborsClassifier(n_neighbors=10))])

sgd_clf.fit(X_train, y_train)
knb_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('knb_clf', KNeighborsClassifier(n_neighbors=10))])

#### SGDClassifier results

In [36]:
predicted_sgd = sgd_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

               precision    recall  f1-score   support

       Health       0.58      0.70      0.63        37
 Homeless cat       0.50      0.73      0.59        15
  No category       0.87      0.71      0.78       195
      Science       0.33      0.50      0.40         8
Show business       0.59      0.73      0.65        22
   Technology       0.59      0.62      0.61        16
        Video       0.12      0.33      0.17         6

     accuracy                           0.69       299
    macro avg       0.51      0.62      0.55       299
 weighted avg       0.75      0.69      0.71       299



#### KNeighborsClassifier results

In [37]:
predicted_knb = knb_clf.predict(X_test)
print(metrics.classification_report(predicted_knb, y_test))

               precision    recall  f1-score   support

       Health       0.42      0.56      0.48        34
 Homeless cat       0.50      0.92      0.65        12
  No category       0.91      0.64      0.75       225
      Science       0.08      1.00      0.15         1
Show business       0.37      0.83      0.51        12
   Technology       0.41      0.54      0.47        13
        Video       0.06      0.50      0.11         2

     accuracy                           0.65       299
    macro avg       0.39      0.71      0.45       299
 weighted avg       0.78      0.65      0.69       299



In this task SGDClassifier also worked better then KNeighborsClassifier. so I take SGDClassifier as a model for other categories classification. Firstly I optimize its hyperparameters.

In [38]:
sgd_clf = Pipeline(steps=[
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42))])

parameters = {
    'sgd_clf__loss': ['hinge', 'modified_huber'], 
    'sgd_clf__penalty': ['l2', 'l1', 'elasticnet']
}

clf = GridSearchCV(sgd_clf, parameters, n_jobs=5)
clf.fit(X_train, y_train)

print("Best parameter (CV score=%0.3f):" % clf.best_score_)
print(clf.best_params_)

Best parameter (CV score=0.672):
{'sgd_clf__loss': 'hinge', 'sgd_clf__penalty': 'l1'}


In [39]:
other_categ_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42, loss='hinge', penalty='l1'))])

other_categ_clf.fit(X_train, y_train)

predicted_other_categ_clf = other_categ_clf.predict(X_val)
print(metrics.classification_report(predicted_other_categ_clf, y_val))

               precision    recall  f1-score   support

       Health       0.38      0.56      0.45        27
 Homeless cat       0.60      0.52      0.56        23
  No category       0.79      0.70      0.74       147
      Science       0.27      0.75      0.40         4
Show business       0.77      0.67      0.71        15
   Technology       0.27      0.33      0.30        12
        Video       0.36      0.36      0.36        11

     accuracy                           0.63       239
    macro avg       0.49      0.56      0.50       239
 weighted avg       0.67      0.63      0.65       239



Save model to pickle file:

In [40]:
with open("other_categ_clf.pkl", "wb") as file: 
    pickle.dump(other_categ_clf, file) 

Average F4 score is 0.50 for 7 categories classification task. "Technology" and "Video" categories  are the worst predicted categories.

### 4) Сombining two classificators together

Now I combine two stages - humor classification and other categories classification - in one system and test them together on new data. For testing these two stages together I have data, which i saved earlier. These saved data were not used in training and testing of separate models. 

### Prediction and results pre processing

In [41]:
X_test = final_test_dataset['cleaned_text']

predicted_humor_clf = humor_clf.predict(X_test)
predicted_other_categ_clf = other_categ_clf.predict(X_test)

pred_humor = []
for i in predicted_humor_clf:
    pred_humor.append(str(i))

pred_other_categ = []
for i in predicted_other_categ_clf:
    pred_other_categ.append(str(i))

    
combine_results = [a +','+ b for a, b in zip(pred_humor, pred_other_categ)]
combine_results = [str_categ.split(',') for str_categ in combine_results]
combine_results = pd.Series(combine_results)


In [42]:
def remove_additional_categories(categories):
    categ = [i for i in categories if i != 'No category']
    categ = [i for i in categ if i != 'not Humor']
    return ', '.join(categ)
       
combine_results_no_category_removed = combine_results.apply(lambda category: remove_additional_categories(category))

### Comparison of results

In [43]:
number_of_full_true_answers = 0
final_test_dataset['cleaned_category_sets']
for idx_ct, category_true in enumerate(final_test_dataset['cleaned_category_sets']):
    for idx_p, category_pred in enumerate(combine_results_no_category_removed):
        if idx_ct == idx_p:
            if category_true == category_pred:
                number_of_full_true_answers += 1

number_of_partly_true_answers = 0              
final_test_dataset['cleaned_category_sets']
for idx_ct, category_true in enumerate(final_test_dataset['cleaned_category_sets']):
    for idx_p, category_pred in enumerate(combine_results_no_category_removed):
        if idx_ct == idx_p:
            category_true_list = category_true.split(',')
            category_pred_list = category_pred.split(',')
            for categ in category_pred_list:
                if categ in category_true_list:
                    number_of_partly_true_answers += 1
                    break

### Results

In [44]:
print(str(number_of_full_true_answers) + ' of ' + str(len(final_test_dataset)) + ' category sets were predicted completely right')
print(str(number_of_partly_true_answers) + ' of ' + str(len(final_test_dataset)) + ' predicted category sets has at least one right predicted category.')

242 of 460 category sets were predicted completely right
309 of 460 predicted category sets has at least one right predicted category.


In [63]:
print('Accuracy that all categories were predicted correctly is ', str(round(number_of_full_true_answers/len(final_test_dataset)*100,2)))
print('Accuracy that at least one category was predicted correctly is ', str(round(number_of_partly_true_answers/len(final_test_dataset)*100,2)))

Accuracy that all categories were predicted correctly is  52.61
Accuracy that at least one category was predicted correctly is  67.17


## 3. Sentiment analysis

For sentiment analysis I use pre-trained VADER tool. 

VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media.

I don't train algorithm, I just predict if article is positive, negative or neutral with already pre-tarined tool.

In [46]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#calculate the negative, positive, neutral and compound scores, plus verbal evaluation
def sentiment_vader(sentence):

    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()

    sentiment_dict = sid_obj.polarity_scores(sentence)
    negative = sentiment_dict['neg']
    neutral = sentiment_dict['neu']
    positive = sentiment_dict['pos']
    compound = sentiment_dict['compound']

    if sentiment_dict['compound'] >= 0.05 :
        overall_sentiment = "Positive"

    elif sentiment_dict['compound'] <= - 0.05 :
        overall_sentiment = "Negative"

    else :
        overall_sentiment = "Neutral"
  
    return overall_sentiment



In [47]:
data['sentiment'] = data['text'].apply(lambda text: sentiment_vader(text))

In [48]:
data.head()

,title,link,date,text,category,cleaned_text,categories_without_countries,sentiment
0,Indiana Jones and a Cat,http://catdailynews.com/2023/06/indiana-jones-...,"June 30, 2023",With the latest Indiana Jones movie out in the...,"Humor, Video",with latest indiana jones movie theater people...,"Humor, Video",Positive
1,Interesting Cat Facts,http://catdailynews.com/2023/06/interesting-ca...,"June 29, 2023",Many people love cats. However what makes cat...,Humor,many people love cat however make cat even lov...,Humor,Positive
2,Cat Gets “Jailed” For Being Too Daring,http://catdailynews.com/2023/06/cat-gets-jaile...,"June 28, 2023",Alice likes traveling in a van on the road wit...,"Humor, Travel",alice like traveling van road cat unfortunatel...,"Humor, Travel",Positive
3,Toys to Keep Your Cat Happy,http://catdailynews.com/2023/06/toys-to-keep-y...,"June 27, 2023","Cats need lots of attention, except when they ...",Humor,cat need lot attention except want left alone ...,Humor,Positive
4,The Evolution of Cats,http://catdailynews.com/2023/06/the-evolution-...,"June 26, 2023",People love cats so it’s no surprise that peop...,"Science, Egypt",people love cat it’s surprise people also stud...,Science,Positive


In [49]:
print('Number of positive cat news out of all news', str(len(data[data['sentiment'] == "Positive"])) + '/' + str(len(data)))
print('Number of negative cat news out of all news', str(len(data[data['sentiment'] == "Negative"])) + '/' + str(len(data)))
print('Number of neutral cat news out of all news', str(len(data[data['sentiment'] == "Neutral"])) + '/' + str(len(data)))
print('Cat news are mostly positive.')

Number of positive cat news out of all news 1600/1990
Number of negative cat news out of all news 364/1990
Number of neutral cat news out of all news 26/1990
Cat news are mostly positive.


## 4. Web-application creation

In [50]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.display import display, clear_output

In [51]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import string
import re
import nltk
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

import pickle

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\polin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\polin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [52]:
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix
# !jupyter serverextension enable voila --sys-prefix

In [55]:
def read_news_from_main_web_page():
    news_data = {'Article title': [], 'Link': [], 'text': [], 'cleaned_text': [], 'Date': [], 'Categories': [], 'Predicted categories': [], 'Sentiment': []}
    page = "http://catdailynews.com/"
    protocol_list = requests.get(page).text
    soup = BeautifulSoup(protocol_list, 'lxml')
    
    for ai in soup.find_all('a'):
        if ai.div:
            news_data['Article title'].append(ai.div.text)
            news_data['Link'].append(ai.get('href'))
    return news_data

def remove_stopwords(text):
    return " ".join([w if w not in eng_stopwords else ' ' for w in text.split(' ')])

def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

def remove_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text, flags=re.I)

def lemmatize_text(text):
    return " ".join([lemma.lemmatize(word.lower()) for word in text.split(' ')])

def news_text_cleaning(text):
    text_removed_stopwords = remove_stopwords(text)
    text_removed_punctuation = remove_punctuation(text_removed_stopwords)
    text_removed_numbers = remove_numbers(text_removed_punctuation)
    text_removed_multiple_spaces = remove_multiple_spaces(text_removed_numbers)
    text_cleaned = lemmatize_text(text_removed_multiple_spaces)
    return text_cleaned

def predict_humor(text):
    return humor_clf.predict([text])

def predict_other_categories(text):
    return other_categ_clf.predict([text])

def predict_categories(text):
    predicted_humor = predict_humor(text)
    predicted_humor = str(predicted_humor[0])
    predicted_other_categ = predict_other_categories(text)
    predicted_other_categ = str(predicted_other_categ[0])
    if predicted_humor == 'not Humor':
        return predicted_other_categ
    elif predicted_humor == 'Humor' and predicted_other_categ == 'No category':
        return predicted_humor
    else:
        return predicted_humor + ', ' + predicted_other_categ

lemma = WordNetLemmatizer()

eng_stopwords = stopwords.words("english")
eng_stopwords.extend(['…', '«', '»', '...'])

with open("humor_clf.pkl", "rb") as file:
    humor_clf = pickle.load(file)
    
with open("other_categ_clf.pkl", "rb") as file:
    other_categ_clf = pickle.load(file)
    
def sentiment_vader(sentence):

    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()

    sentiment_dict = sid_obj.polarity_scores(sentence)
    negative = sentiment_dict['neg']
    neutral = sentiment_dict['neu']
    positive = sentiment_dict['pos']
    compound = sentiment_dict['compound']

    if sentiment_dict['compound'] >= 0.05 :
        overall_sentiment = "Positive"

    elif sentiment_dict['compound'] <= - 0.05 :
        overall_sentiment = "Negative"

    else :
        overall_sentiment = "Neutral"
  
    return overall_sentiment

def predict_category(news_data):
    for idx, link in enumerate(news_data['Link']):
        protocol_list = requests.get(link).text
        soup = BeautifulSoup(protocol_list, 'lxml')

        text_parts = soup.find_all('p')
        text = ''
        for p in text_parts:
            text += p.text
        news_data['text'].append(text)
        text_cleaned = news_text_cleaning(text)
        news_data['cleaned_text'].append(text_cleaned)

        for divi in soup.find_all('div'):
            if divi.get("class") == ['pull-left']:
                date = divi.text.split('  ')[0][:-2]
        news_data['Date'].append(date)

        categories = []
        for divi in soup.find_all('div'):
            if divi.get("class") == ['category-tag']:
                all_a = divi.find_all('a')
                for a in all_a:
                    if a.text[0].isupper():
                        categories.append(a.text)
                categories = ', '.join(categories)
        news_data['Categories'].append(categories)

        pred_categories = predict_categories(text_cleaned)
        news_data['Predicted categories'].append(pred_categories)
        
        pred_sentiment = sentiment_vader(text_cleaned)
        news_data['Sentiment'].append(pred_sentiment)
    return news_data


def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

def create_news_table_show(news_data):
    news_data_table = pd.DataFrame(news_data)
    news_table_show = news_data_table[['Date', 'Article title', 'Link', 'Categories', 'Predicted categories', 'Sentiment']]
    blankIndex=[''] * len(news_table_show)
    news_table_show.index=blankIndex
    news_table_show = news_table_show.style.format({'Link': make_clickable})
    return news_table_show


In [56]:
# Image Widget

file = open("cat_daily_news_main.jpg", "rb")
main_image = file.read()

file = open("sleepy_kitten.jpg", "rb")
lovely_image = file.read()

image_headline = widgets.Image(
                    value=main_image,
                    format='jpg',
                    width='95%',
                    height='500px'
                )

image_base = widgets.Image(
                    value=lovely_image,
                    format='jpg',
                    width='350px',
                )

label_base = widgets.Label(
                    value='Photo by fanibani',
                    style={'description_width': 'initial'}
                )


vbox_headline_image = widgets.VBox([image_headline])
vbox_base_image = widgets.VBox([image_base, label_base])
# display(vbox_headline_image, vbox_base_image)

In [57]:
# button to load news

button_send = widgets.Button(
                description='Load news',
                tooltip='Send',
                style={'description_width': 'initial'}
            )

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        news_data = read_news_from_main_web_page()
        news_data_predicted = predict_category(news_data)
        news_table_show = create_news_table_show(news_data_predicted)
        display(news_table_show)
        
button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send, output])
# display(vbox_result)

In [58]:
# news table widget

# def news_table_widget_creation(df):
    
#     out = widgets.Output(layout={'border': '1px solid black'})
#     with out:
#         display(df)
#     return out

# news_table = news_table_widget_creation(news_table_show)
# vbox_news_table = widgets.VBox([news_table])
# display(vbox_news_table)

In [59]:
level1 = widgets.HBox([vbox_headline_image])
level2 = widgets.HBox([vbox_base_image, vbox_result])
# level2 = widgets.HBox([vbox_result])
# level2 = widgets.HBox([vbox_images, vbox_result])
display(level1,level2)

Then I appload Web-application code to GitHub and make an application using Mybinder.org. 